In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
df = pd.read_csv("advertising.csv")
df = df.iloc[:,1:len(df)]
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [3]:
X = df.drop("sales", axis = 1)
y = df[["sales"]]

In [4]:
X.head()

,TV,radio,newspaper
0,230.1,37.8,69.2
1,44.5,39.3,45.1
2,17.2,45.9,69.3
3,151.5,41.3,58.5
4,180.8,10.8,58.4


In [5]:
y.head()

,sales
0,22.1
1,10.4
2,9.3
3,18.5
4,12.9


## 1. StatsModels

* **R² (R-squared):** Bağımsız değişkenlerin bağımlı değişkendeki değişikliği açıklama yüzdesidir.
* Burada açıklama yüzdesinin oldukça iyi olduğunu görüyoruz.
* **Ajd. R-squared:** R² değerinin düzeltilmiş halidir. Daha iyi bir açıklanabilirlik oranı vermektedir.
* **F-statistic:** 0.05'ten oldukça küçüktür. Modelin anlamlı olduğu bilgisini verir.
* **coef:** Kurulacak olan nihai modeldeki bağımsız değişkenlerin katsayılarını bize vermektedir.
* **std err (standart hata):** Coef katsayılarının standart hatasını vermektedir.
* Elimizdeki katsayıların anlamlı olup olmadığını **t-test** ve **p-value** ile anlarız.
* **P > |t|:** Baktığımız zaman modellemek için kurduğumuz bütün değişkenler anlamlıdır. (0.05'ten küçük ise anlamlı)
* **coef** katsayılırını yorumlarsak:
    * Diğer değişkenler sabit varsayıldığı zaman 
    * TV harcamalarında meydana gelecek 1 birimlik artış
    * Bağımlı değişkenimiz **sales** değişkeninde
    * Ortalama 0.05'lik birimlik bir artışa sebep olacaktır.

In [6]:
model = sm.OLS(y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  sales   R-squared (uncentered):                   0.982
Model:                            OLS   Adj. R-squared (uncentered):              0.982
Method:                 Least Squares   F-statistic:                              3566.
Date:                Sat, 17 Apr 2021   Prob (F-statistic):                   2.43e-171
Time:                        17:28:37   Log-Likelihood:                         -423.54
No. Observations:                 200   AIC:                                      853.1
Df Residuals:                     197   BIC:                                      863.0
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TV             0.0538      0.001     40.507      0.000       0.051       0.056
radio          0.2222      0.009     23.595      0.000       0.204       0.241
newspaper      0.0168      0.007      2.517      0.013       0.004       0.030
==============================================================================
Omnibus:                        5.982   Durbin-Watson:                   2.038
Prob(Omnibus):                  0.050   Jarque-Bera (JB):                7.039
Skew:                          -0.232   Prob(JB):                       0.0296
Kurtosis:                       3.794   Cond. No.                         12.6
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 2. Scikit-Learn Model

In [7]:
model = LinearRegression().fit(X,y)
model

LinearRegression()

* **y:** Bağımlı Değişkeni

In [8]:
model.intercept_

array([2.93888937])

* **X:** Bağımsız Değişkenleri

In [9]:
model.coef_

array([[ 0.04576465,  0.18853002, -0.00103749]])

### Denklem

* **Sales** = ```2.94``` + ```TV*0.04``` + ```radio*0.19``` - ```newspaper*0.001```

### Hata Kareler Ortalaması

* Fonksiyon içerisine gerçek y değerlerini ve yeni tahmin değerlerimizi koyuyoruz.

In [10]:
MSE = mean_squared_error(y, model.predict(X))
MSE

2.784126314510936

### Hata Kareler Ortalamasının Karekökü

In [11]:
RMSE = np.sqrt(MSE)
RMSE

1.6685701407225697

### Model Doğrulama (Model Tuning)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)
model = LinearRegression().fit(X_train, y_train)

* Eğitim Hatası

In [13]:
np.sqrt(mean_squared_error(y_train, model.predict(X_train)))

1.5513912323520802

* Test Hatası

In [14]:
np.sqrt(mean_squared_error(y_test, model.predict(X_test)))

2.0981225634956804

### Cross-Validation

In [15]:
cross_val_score(model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")

array([-1.71524789, -1.67158319, -1.83164463, -4.44175358, -1.22520234,
       -2.78857297, -3.57788547, -2.46388059, -4.00592555, -2.39062693])

In [16]:
np.mean(-cross_val_score(model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error"))

2.6112323155583232